In [1]:
from gabriel_lego import LEGOTask
from tqdm.notebook import tqdm

import edgedroid.data as e_data
from edgedroid.models import *
from generate_corr_sequence import gen_corr_sequence
import scipy.stats as st

model = EdgeDroidModel(
    frame_trace=e_data.load_default_trace("square00", truncate=5),
    frame_model=ZeroWaitSamplingPolicy.from_default_data(),
    timing_model=MultiCurveFittingExecutionTimeModel(neuroticism=1.0)
)

task = LEGOTask(e_data.load_default_task("square00", truncate=5))


acf_50 = 1 / (2 ** np.arange(5))  # 0.5 corr
acf_25 = 1 / (4 ** np.arange(5))
acf_12 = 1 / (8 ** np.arange(5))
acf_00 = np.zeros(5)
acf_00[0] = 1.0

num_frames_to_generate = 7200 * 30  # assume task takes 2hrs at 30fps 


def emulate_run(model: EdgeDroidModel, rtt_s_iter: Iterator[float]):
    for model_step in model.play_steps_emulated_time():
        # in this setup, we have rtts per frame
        frame_timings: Optional[FrameTimings] = None
        prev_frame: Optional[str] = None
        while True:
            try:
                model_frame = model_step.send(frame_timings)
            except StopIteration:
                if not prev_frame or prev_frame.lower() not in ("success", "initial"):
                    raise Exception(prev_frame)
                break

            # prev_result = task.submit_frame(model_frame.frame_data)
            prev_frame = model_frame.frame_tag
            frame_rtt = next(rtt_s_iter)
            frame_timings = FrameTimings(frame_rtt * 0.9, frame_rtt * 0.1)

delays  = gen_corr_sequence(
    dist_obj=st.lognorm(s=1.2, loc=0.1, scale=0.1),
    L=num_frames_to_generate,
    target_acf=acf_50,
    debug=False,
)

emulate_run(model, iter(delays))
model.model_step_metrics()

,last_frame_rtt,execution_time,step_duration,time_to_feedback,wait_time,frame_count
step_number,,,,,,
0,0.127882,0.000000,0.127882,0.127882,0.000000,1
1,0.230002,4.998234,5.241863,0.243629,0.013626,28
2,0.168279,5.042515,5.251912,0.209398,0.041119,25
3,0.620594,5.029906,5.923346,0.893440,0.272846,11
4,0.407016,5.217898,5.627218,0.409321,0.002305,10
5,0.517605,5.098132,5.660805,0.562672,0.045068,29


In [ ]:
import itertools as it
import multiprocess as mp
from typing import Tuple

trace = "square00"
truncate = 50
reps_per_model = 30

def lognorm_with_params(loc: float | int, mean: float | int, var: float | int) -> Tuple[stats.rv_continuous, float, float]:
    sigma2 = np.log((var / np.square(mean - loc)) + 1)
    mu = np.log(mean - loc) - (sigma2 / 2)
    scale = np.exp(mu)
    s = np.sqrt(sigma2)
    return st.lognorm(loc=loc, scale=scale, s=s), scale, s

def gamma_with_params(loc: float | int, mean: float | int, var: float | int) -> Tuple[stats.rv_continuous, float, float]:
    scale = var / (mean - loc)
    k = np.square(mean - loc) / var
    return st.gamma(a=k, loc=loc, scale=scale), scale, k

class ThirdDegPolyExecutionTimeModel(MultiCurveFittingExecutionTimeModel):
    _fit_functions = (CubeFit,)


timing_frame_models = {
    "3rd-poly-high": lambda: (ThirdDegPolyExecutionTimeModel(neuroticism=1.0), ZeroWaitSamplingPolicy.from_default_data()),
    "3rd-poly-low": lambda:(ThirdDegPolyExecutionTimeModel(neuroticism=0.0), ZeroWaitSamplingPolicy.from_default_data()),
    "legacy": lambda: (LegacyModel(), LegacySamplingPolicy.from_default_data()),
    "first-order": lambda: (FirstOrderETM(), ZeroWaitSamplingPolicy.from_default_data()),
    "first-order-median": lambda: (FirstOrderAggregateETM(np.median), ZeroWaitSamplingPolicy.from_default_data())
}

min_bound = 0.042  # 24FPS

rhos = (
    (0.0, acf_00),
    # (0.125, acf_12),
    (0.250, acf_25),
    (0.500, acf_50),
)

gamma_dists = (
    gamma_with_params(loc=min_bound, mean=2*0.05, var=2*0.01),
    gamma_with_params(loc=min_bound, mean=2*0.10, var=2*0.02),
    gamma_with_params(loc=min_bound, mean=2*0.20, var=2*0.04),
    gamma_with_params(loc=min_bound, mean=2*0.40, var=2*0.08),
    gamma_with_params(loc=min_bound, mean=2*0.80, var=2*0.16),
    gamma_with_params(loc=min_bound, mean=2*0.16, var=2*0.32),
)

def run_model(params):
    rep, (rho, acf_fn), (gamma, scale, k), (model_name, model_constructor) = params
    timing_model, sampling_policy = model_constructor()

    model = EdgeDroidModel(
        frame_trace=e_data.load_default_trace(trace, truncate=truncate),
        frame_model=sampling_policy,
        timing_model=timing_model
    )

    delays = gen_corr_sequence(
        dist_obj=gamma,
        L=num_frames_to_generate,
        target_acf=acf_fn,
        debug=False,
    )

    emulate_run(model, iter(delays))

    d_mean, d_var, *_ = gamma.stats()

    df = model.model_step_metrics()
    df["model"] = model_name
    df["rep"] = rep
    df["rho"] = rho
    df["gamma_loc"] = min_bound
    df["gamma_scale"] = scale
    df["gamma_shape"] = k
    df["gamma_mean"] = d_mean
    df["gamma_var"] = d_var
    
    return df

params_iter = list(it.product(
    range(reps_per_model), 
    rhos, 
    gamma_dists,
    timing_frame_models.items(),
))
tqdm_iter = iter(tqdm(params_iter, desc="Running models"))

dfs = deque()

with mp.Pool() as pool:
    for df in pool.imap_unordered(run_model, params_iter): 
        dfs.append(df)
        next(tqdm_iter)

data = pd.concat(dfs)
data.to_csv("./per_frame_delay_new_gamma_params.csv")
data

Running models:   0%|          | 0/2700 [00:00<?, ?it/s]

/Users/molguin/.venvs/EdgeDroid/lib/python3.11/site-packages/generate_corr_sequence/generate_corr_sequence.py:46: IntegrationWarning: The algorithm does not converge.  Roundoff error is detected
  in the extrapolation table.  It is assumed that the requested tolerance
  cannot be achieved, and that the returned result (if full_output = 1) is 
  the best which can be obtained.
  I = quad(integration_function, integration_min, integration_max, args=(numberOfHermitePoly, dist_obj))
/Users/molguin/.venvs/EdgeDroid/lib/python3.11/site-packages/generate_corr_sequence/generate_corr_sequence.py:46: IntegrationWarning: The algorithm does not converge.  Roundoff error is detected
  in the extrapolation table.  It is assumed that the requested tolerance
  cannot be achieved, and that the returned result (if full_output = 1) is 
  the best which can be obtained.
  I = quad(integration_function, integration_min, integration_max, args=(numberOfHermitePoly, dist_obj))
/Users/molguin/.venvs/EdgeDroid/